In [1]:
import torch
from tqdm import tqdm
from src.utils.mlflow_utils import log_metrics, log_params, log_model
from src.data.data_preprocessing import ProjectDataPreprocessing
from src.models.model_architecture import LenetModel
import yaml

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

In [8]:
data_processor = ProjectDataPreprocessing()

In [9]:
train_loader, test_loader, val_loader = data_processor.preprocess_data(config)

In [5]:
next(iter(train_loader))[0].shape

torch.Size([32, 3, 224, 224])

In [6]:
next(iter(train_loader))[1].shape

torch.Size([32, 2])

In [4]:
model = LenetModel(config)

In [5]:
from torchinfo import summary

summary(model, input_size=(32, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
LenetModel                               [32, 2]                   --
├─Sequential: 1-1                        [32, 2]                   --
│    └─Conv2d: 2-1                       [32, 32, 222, 222]        2,432
│    └─Tanh: 2-2                         [32, 32, 222, 222]        --
│    └─AvgPool2d: 2-3                    [32, 32, 111, 111]        --
│    └─Conv2d: 2-4                       [32, 64, 109, 109]        51,264
│    └─Tanh: 2-5                         [32, 64, 109, 109]        --
│    └─AvgPool2d: 2-6                    [32, 64, 54, 54]          --
│    └─Conv2d: 2-7                       [32, 128, 52, 52]         204,928
│    └─Tanh: 2-8                         [32, 128, 52, 52]         --
│    └─AvgPool2d: 2-9                    [32, 128, 26, 26]         --
│    └─Dropout: 2-10                     [32, 128, 26, 26]         --
│    └─Flatten: 2-11                     [32, 86528]               --
│  

In [15]:
def main(config, model, train_loader, test_loader):
    """
    Trains input model given dataloaders, optimizer and loss function,
    Tracking with MLFlow
    """
    with mlflow.start_run():
        mlflow.set_experiment("object-detection-project")
        log_params(config)

        for epoch in tqdm(range(config['train']['num_epochs']), desc = 'Model training..' ):
            loss = 0
            train_loss = 0
            for image, label in train_loader:
                image = image.to(device)
                label = label.to(device)
                optimizer.zero_grad()
                model.train()
                outputs = model(image)
                loss = loss_function(outputs, label)
                loss.backward()
                optimizer.step()
                train_loss += loss * inputs.size(0)
            train_loss=train_loss/len(train_loader.dataset)

            loss = 0
            val_loss = 0
            with torch.inference_mode():
                for image, label in val_loader:
                    image = image.to(device)
                    label = label.to(device)
                    model.eval()
                    outputs = model(image)
                    loss = loss_function(outputs, label)
                    val_loss += loss * inputs.size(0)
                val_loss=val_loss/len(val_loader.dataset)
                if val_loss < best_loss:
                    print('Saving best model')
                    torch.save(model.state_dict(), '../../models/{}.pt'.format(model_name))
                    best_loss = val_loss
            print("Epoch: {}/{}".format(epoch, config['train']['epoch']))
            print("Train loss: {}")
            log_metrics({"train_loss": train_loss,
                         "val_loss": val_loss})
            log_model(model, "model")
            return model

In [11]:
import mlflow

In [16]:
main(config, model, train_loader, test_loader)

Model training..:   0%|          | 0/50 [00:01<?, ?it/s]


NameError: name 'optimizer' is not defined